In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(optimizer=keras.optimizers.RMSprop(0.1), loss='mean_squared_error', metrics='mean_absolute_error')
    return model

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print('Starting training; got log keys: {}'.format(keys))
    
    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print('Stoping training; got log keys: {}'.format(keys))
    
    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print('Start epoch {}, got log keys: {}'.format(epoch, keys))
    
    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print('End epoch {}, got log keys: {}'.format(epoch, keys))
    
    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print('Starting testing; got log keys: {}'.format(keys))
    
    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print('Stoping testing; got log keys: {}'.format(keys))
    
    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print('Starting predicting; got log keys: {}'.format(keys))
    
    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print('Stoping predicting; got log keys: {}'.format(keys))
    
    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print('...Training: start of batch {}; got log keys: {}'.format(batch, keys))
    
    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print('...Training: end of batch {}; got log keys: {}'.format(batch, keys))
    
    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print('...Testing: start of batch {}; got log keys: {}'.format(batch, keys))
    
    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print('...Testing: end of batch {} got log keys: {}'.format(batch, keys))
    
    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print('...Predicting: start of batch {}; got log keys: {}'.format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print('...Predicting: end of batch {}; got log keys: {}'.format(batch, keys))

In [ ]:
model = get_model()

history = model.fit(x_train, y_train, batch_size=128, epochs=1, verbose=0, validation_split=0.5, callbacks=[CustomCallback()])

res = model.evaluate(x_test, y_test, batch_size=128, verbose=0, callbacks=[CustomCallback()])

res = model.predict(x_test, batch_size=128, verbose=0, callbacks=[CustomCallback()])

In [ ]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))
    
    def on_test_batch_end(self, batch, logs=None):
        print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))
    
    def on_epoch_end(self, epoch, logs=None):
        print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}'.format(epoch, logs['loss'], logs['mean_absolute_error']))

model = get_model()

history = model.fit(x_train, y_train, batch_size=128, verbose=0, epochs=2, callbacks=[LossAndErrorPrintingCallback()])

res = model.evaluate(x_test, y_test, batch_size=128, verbose=0, callbacks=[LossAndErrorPrintingCallback()])

In [ ]:
import numpy as np

class EarlyStopingAtMinLoss(keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(EarlyStopingAtMinLoss, self).__init__()
        self.patience = patience
        self.best_weights = None
    
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
    
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get('loss')
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('Restoring model weights from the end of the best epoch')
                self.model.set_weights(self.best_weights)
    
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

model = get_model()

model.fit(x_train, y_train, batch_size=64, steps_per_epoch=5, epochs=30, verbose=0, callbacks=[LossAndErrorPrintingCallback(), EarlyStopingAtMinLoss()])

In [ ]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule
    
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        scheduled_lr = self.schedule(epoch, lr)

        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print('\nEpoch %05d: Learning rate is %6.4f' % (epoch, scheduled_lr))

LR_SCHEDULE = [
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001)
]

def lr_schedule(epoch, lr):
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][1]:
            return LR_SCHEDULE[i][1]
        
    return lr


model = get_model()
model.fit(x_train, y_train, batch_size=64, steps_per_epoch=5, epochs=15, verbose=0, callbacks=[LossAndErrorPrintingCallback(), CustomLearningRateScheduler(lr_schedule)])